# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,16.42,68,38,8.86,SH,1706563102
1,1,tralee,52.2704,-9.7026,3.85,77,7,2.51,IE,1706563103
2,2,iqaluit,63.7506,-68.5145,-30.73,67,100,0.00,CA,1706563104
3,3,port-aux-francais,-49.3500,70.2167,3.94,70,99,16.61,TF,1706563104
4,4,mount pearl,47.5166,-52.7813,-2.25,95,100,8.23,CA,1706563163


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM"
)

# Display the map
map_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
non_cloudy_df = city_data_df.loc[city_data_df["Cloudiness"]==0, :]

# Drop any rows with null values
non_cloudy_df.dropna

# Display sample data
non_cloudy_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,ushuaia,-54.8000,-68.3000,13.81,51,0,11.32,AR,1706563166
8,8,sabha,27.0377,14.4283,10.74,56,0,3.65,LY,1706563166
10,10,sultanah,24.4926,39.5857,19.25,30,0,1.54,SA,1706563168
11,11,aoulef,26.9667,1.0833,18.21,22,0,6.84,DZ,1706563168
27,27,hassi messaoud,31.6804,6.0729,13.92,67,0,2.57,DZ,1706563177
...,...,...,...,...,...,...,...,...,...,...
535,535,hilo,19.7297,-155.0900,30.16,65,0,2.06,US,1706563455
536,536,adrar,20.5022,-10.0711,18.65,13,0,4.69,MR,1706563456
545,545,zouerate,22.7187,-12.4521,19.81,12,0,4.07,MR,1706563281
546,546,boise,43.6135,-116.2035,13.45,66,0,0.00,US,1706563360


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = non_cloudy_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,ushuaia,AR,-54.8000,-68.3000,51,
8,sabha,LY,27.0377,14.4283,56,
10,sultanah,SA,24.4926,39.5857,30,
11,aoulef,DZ,26.9667,1.0833,22,
27,hassi messaoud,DZ,31.6804,6.0729,67,
...,...,...,...,...,...,...
535,hilo,US,19.7297,-155.0900,65,
536,adrar,MR,20.5022,-10.0711,13,
545,zouerate,MR,22.7187,-12.4521,12,
546,boise,US,43.6135,-116.2035,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel",
    "lang":"en"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
sabha - nearest hotel: فندق الكاظم-سبها
sultanah - nearest hotel: فندق جلنار
aoulef - nearest hotel: دار الضياف
hassi messaoud - nearest hotel: AMC
raahe - nearest hotel: Aparthotel Forenom
timimoun - nearest hotel: أناف
cox's bazar - nearest hotel: Hotel Taj Sheba
lagoa - nearest hotel: Casa da Madrinha
hawaiian paradise park - nearest hotel: No hotel found
punta arenas - nearest hotel: Hotel Lacolet
carnarvon - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
north bend - nearest hotel: The Mill Casino Hotel
sept-iles - nearest hotel: Château Arnaud
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
kodiak - nearest hotel: Shelikof Lodge
saint-francois - nearest hotel: Chez Lily
elk city - nearest hotel: Holiday Inn Express & Suites
tura - nearest hotel: No hotel found
greenville - nearest hotel: Quality Inn
chibougamau - nearest hotel: No hotel found
wynyard - nearest hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
7,ushuaia,AR,-54.8000,-68.3000,51,Apart Hotel Aires del Beagle
8,sabha,LY,27.0377,14.4283,56,فندق الكاظم-سبها
10,sultanah,SA,24.4926,39.5857,30,فندق جلنار
11,aoulef,DZ,26.9667,1.0833,22,دار الضياف
27,hassi messaoud,DZ,31.6804,6.0729,67,AMC
...,...,...,...,...,...,...
535,hilo,US,19.7297,-155.0900,65,Dolphin Bay Hotel
536,adrar,MR,20.5022,-10.0711,13,No hotel found
545,zouerate,MR,22.7187,-12.4521,12,فندق تيرس زمور
546,boise,US,43.6135,-116.2035,66,The Grove Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
# https://stackoverflow.com/questions/59678780/show-extra-columns-when-hovering-in-a-scatter-plot-with-hvplot
map_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)